In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
import os
import numpy as np
from scipy.misc import imread, imresize
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

%matplotlib inline

In [3]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [4]:
ALL_DATA_DIR = '/home/datasets/events/'
INPUT_SIZE = (224, 224)
#INPUT_SIZE = (299, 299)

---

# PEC

In [5]:
PEC_DATA_DIR=ALL_DATA_DIR+'pec/'
PEC_TRAIN_DATA_DIR = PEC_DATA_DIR+'full_res/train_all'
PEC_VAL_DATA_DIR = PEC_DATA_DIR+'full_res/val'
PEC_IMG_TRAIN_DATA_DIR = PEC_DATA_DIR+str(INPUT_SIZE[0])+'/train_all'
PEC_IMG_VAL_DATA_DIR = PEC_DATA_DIR+str(INPUT_SIZE[0])+'/val'

if False:
    PEC_TRAIN_DATA_DIR = PEC_IMG_TRAIN_DATA_DIR
    PEC_VAL_DATA_DIR = PEC_IMG_VAL_DATA_DIR

IMG_PEC_DATA_DIR = PEC_DATA_DIR+'images/'
PEC_TRAIN_FILE=PEC_DATA_DIR+'meta/train_all.txt' #'meta/train.txt'
PEC_TEST_FILE=PEC_DATA_DIR+'meta/test.txt'

In [32]:
def save_links(filename, input_dir,dir_to_save):
    lines = [line.strip() for line in open(filename,'r')]
    for row in log_progress(lines):
        category,album=row.split('/')
        src_dir=os.path.join(PEC_DATA_DIR,row)
        outdir_path=os.path.join(dir_to_save,category)
        if not os.path.exists(outdir_path):
            os.makedirs(outdir_path)
        img_files = (f for f in os.listdir(src_dir) if f.endswith('jpg'))
        for f in img_files:
            src_file_path=os.path.join(src_dir,f)
            dst_file_path=os.path.join(outdir_path,album+'_'+f)
            #print(src_file_path,dst_file_path)
            if not os.path.exists(dst_file_path):
                os.symlink(src_file_path,dst_file_path)
        #break

In [33]:
save_links(PEC_TRAIN_FILE,IMG_PEC_DATA_DIR,PEC_TRAIN_DATA_DIR)

In [20]:
save_links(PEC_TEST_FILE,IMG_PEC_DATA_DIR,PEC_VAL_DATA_DIR)

In [36]:
from PIL import Image

def save_images(filename, input_dir,dir_to_save):
    lines = [line.strip() for line in open(filename,'r')]
    for row in log_progress(lines):
        category,album=row.split('/')
        src_dir=os.path.join(PEC_DATA_DIR,row)
        outdir_path=os.path.join(dir_to_save,category)
        if not os.path.exists(outdir_path):
            os.makedirs(outdir_path)
        img_files = (f for f in os.listdir(src_dir) if f.endswith('jpg'))
        for f in img_files:
            src_file_path=os.path.join(src_dir,f)
            dst_file_path=os.path.join(outdir_path,album+'_'+f)
            #print(src_file_path,dst_file_path)
            if not os.path.exists(dst_file_path):
                img=Image.open(src_file_path)
                area = img.resize(INPUT_SIZE)
                area.save(dst_file_path, 'jpeg')
        #break

In [37]:
save_images(PEC_TRAIN_FILE,PEC_DATA_DIR,PEC_IMG_TRAIN_DATA_DIR)

In [38]:
save_images(PEC_TEST_FILE,PEC_DATA_DIR,PEC_IMG_VAL_DATA_DIR)

# CUFED

In [6]:
CUFED_DATA_DIR=ALL_DATA_DIR+'CUFED/'
CUFED_TRAIN_DATA_DIR =CUFED_DATA_DIR+'train'
CUFED_VAL_DATA_DIR = CUFED_DATA_DIR+'val'

IMG_CUFED_DATA_DIR = CUFED_DATA_DIR+'images/'
CUFED_JSON = CUFED_DATA_DIR+'event_type.json'

In [44]:
import json
event2album_train,event2album_val={},{}
with open(CUFED_JSON) as json_file:
    data = json.load(json_file)
    print(len(data))
    end=len(data)-len(data)//5
    for i,album in enumerate(data):
        #print(album,data[album])
        #event=data[album][0]
        for event in [data[album][0]]:
            if i<end:
                if event not in event2album_train:
                    event2album_train[event]=[]
                event2album_train[event].append(album)
            else:
                if event not in event2album_val:
                    event2album_val[event]=[]
                event2album_val[event].append(album)
        #break
print('Train:',len(event2album_train))
total=0
for event in event2album_train:
    print(event,len(event2album_train[event]))
    total+=len(event2album_train[event])
print('total albumes:',total)
    
print('\nval:',len(event2album_val))
total=0
for event in event2album_val:
    print(event,len(event2album_val[event]))
    total+=len(event2album_val[event])
print('total albumes:',total)

1883
1_63438723@N03 ['NatureTrip']
Train: 23
NatureTrip 37
Museum 39
Architecture 36
Protest 38
Graduation 139
UrbanTrip 88
ReligiousActivity 42
CasualFamilyGather 36
PersonalArtActivity 36
Show 105
Birthday 148
Zoo 86
Christmas 85
PersonalMusicActivity 10
Halloween 78
ThemePark 85
GroupActivity 33
BeachTrip 40
Sports 57
BusinessActivity 43
Wedding 152
PersonalSports 55
Cruise 39
total albumes: 1507

val: 23
Zoo 23
Museum 11
Architecture 5
Protest 11
Graduation 37
ReligiousActivity 6
CasualFamilyGather 4
PersonalArtActivity 4
UrbanTrip 29
Birthday 35
NatureTrip 12
Christmas 18
Wedding 43
Halloween 28
ThemePark 14
GroupActivity 12
BeachTrip 6
Sports 15
BusinessActivity 8
PersonalMusicActivity 1
Show 21
PersonalSports 20
Cruise 13
total albumes: 376


In [49]:
def save_links_events(event2album, dir_to_save):
    for event in event2album:
        print(event,len(event2album[event]))
        for album in event2album[event]:
            album_classes=album+'_'+','.join(data[album])
            src_dir=os.path.join(IMG_CUFED_DATA_DIR,album)
            outdir_path=os.path.join(dir_to_save,event)
            if not os.path.exists(outdir_path):
                os.makedirs(outdir_path)
            img_files = (f for f in os.listdir(src_dir) if f.endswith('jpg') and not f.startswith('.'))
            for f in img_files:
                src_file_path=os.path.join(src_dir,f)
                dst_file_path=os.path.join(outdir_path,album_classes+'_'+f)
                #print(src_file_path,dst_file_path)
                if not os.path.exists(dst_file_path):
                    os.symlink(src_file_path,dst_file_path)
            #break

In [50]:
save_links_events(event2album_train,CUFED_TRAIN_DATA_DIR)

NatureTrip 37
Museum 39
Architecture 36
Protest 38
Graduation 139
UrbanTrip 88
ReligiousActivity 42
CasualFamilyGather 36
PersonalArtActivity 36
Show 105
Birthday 148
Zoo 86
Christmas 85
PersonalMusicActivity 10
Halloween 78
ThemePark 85
GroupActivity 33
BeachTrip 40
Sports 57
BusinessActivity 43
Wedding 152
PersonalSports 55
Cruise 39


In [51]:
save_links_events(event2album_val,CUFED_VAL_DATA_DIR)

Zoo 23
Museum 11
Architecture 5
Protest 11
Graduation 37
ReligiousActivity 6
CasualFamilyGather 4
PersonalArtActivity 4
UrbanTrip 29
Birthday 35
NatureTrip 12
Christmas 18
Wedding 43
Halloween 28
ThemePark 14
GroupActivity 12
BeachTrip 6
Sports 15
BusinessActivity 8
PersonalMusicActivity 1
Show 21
PersonalSports 20
Cruise 13


In [521]:
import json
with open(CUFED_DATA_DIR+'image_importance.json') as json_file:
    data = json.load(json_file)
    print(len(data))
    for album in data:
        prev_name=''
        for image in data[album]:
            _,img_short_name,importance=image
            if img_short_name<prev_name:
                print(album,prev_name,img_short_name,data[album])
                break
            prev_name=img_short_name

1883
1_80547277@N00 1_80547277@N00/289962812 1_80547277@N00/289797323 [['http://farm1.staticflickr.com/116/289796970_94b2967bc6.jpg', '1_80547277@N00/289796970', 1.4], ['http://farm1.staticflickr.com/109/289962812_37b9e71856.jpg', '1_80547277@N00/289962812', 1.0], ['http://farm1.staticflickr.com/106/289797323_507d9e4122.jpg', '1_80547277@N00/289797323', 0.4], ['http://farm1.staticflickr.com/107/289797574_12161d45a8.jpg', '1_80547277@N00/289797574', 0.8], ['http://farm1.staticflickr.com/120/289797947_89717f2959.jpg', '1_80547277@N00/289797947', 0.8], ['http://farm1.staticflickr.com/100/289798178_b771053bef.jpg', '1_80547277@N00/289798178', 0.2], ['http://farm1.staticflickr.com/108/289798475_d5f8e276fe.jpg', '1_80547277@N00/289798475', 0.2], ['http://farm1.staticflickr.com/115/289798690_1a8715d48f.jpg', '1_80547277@N00/289798690', 0.6], ['http://farm1.staticflickr.com/102/289798843_70990eaf13.jpg', '1_80547277@N00/289798843', 1.4], ['http://farm1.staticflickr.com/112/289799000_df2a874419

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [520]:
import json
img_short_name2importance={}
with open(CUFED_DATA_DIR+'image_importance.json') as json_file:
    data = json.load(json_file)
    print(len(data))
    for album in data:
        for image in data[album]:
            _,img_short_name,importance=image
            img_short_name2importance[img_short_name]=float(importance)
            

album2importances_val=[]
for class_name in os.listdir(val_data_dir):
    y=class_to_idx[class_name]
    class_dir=os.path.join(val_data_dir,class_name)
    album2importances={}
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith('jpg'):
            img_name_parts=img_name.split('_')
            album='_'.join(img_name_parts[:-1])
            img_short_name=img_name_parts[0]+'_'+img_name_parts[1]+'/'+img_name_parts[-1][:-4]
            #print(album,img_short_name,img_short_name2importance[img_short_name],img_name)
            if album in album2importances:
                album2importances[album].append(img_short_name2importance[img_short_name])
            else:
                album2importances[album]=[img_short_name2importance[img_short_name]]
            i+=1
    album2importances_val.append(album2importances)

print(len(y_class_val), len(album2importances_val),len(album2importances_val[0]),len(album2indices_val[0]))


album2importances_train=[]
for class_name in os.listdir(train_data_dir):
    y=class_to_idx[class_name]
    class_dir=os.path.join(train_data_dir,class_name)
    album2importances={}
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith('jpg'):
            img_name_parts=img_name.split('_')
            album='_'.join(img_name_parts[:-1])
            img_short_name=img_name_parts[0]+'_'+img_name_parts[1]+'/'+img_name_parts[-1][:-4]
            #print(album,img_short_name,img_short_name2importance[img_short_name],img_name)
            if album in album2importances:
                album2importances[album].append(img_short_name2importance[img_short_name])
            else:
                album2importances[album]=[img_short_name2importance[img_short_name]]
            i+=1
    album2importances_train.append(album2importances)

print(len(y_class_train), len(album2importances_train),len(album2importances_train[0]),len(album2indices_train[0]))

IMPORTANCE_FILE='cufed_aggregation_importance.npz'
np.savez(IMPORTANCE_FILE, album2importances_train, album2importances_val)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



23 23 4 4
23 23 36 36


## Filter 

In [297]:
removed_classes=['GroupActivity', 'PersonalArtActivity','ReligiousActivity','CasualFamilyGather','PersonalMusicActivity']

event2album_train,event2album_val={},{}
with open(CUFED_JSON) as json_file:
    data = json.load(json_file)
    print(len(data))
    end=len(data)-len(data)//5
    for i,album in enumerate(data):
        filtered = [x.lower() for x in data[album] if x not in removed_classes]
        #print(album,data[album],filtered)
        #event=data[album][0]
        if len(filtered)==0:
            continue
        for event in [filtered[0]]:
            if i<end:
                if event not in event2album_train:
                    event2album_train[event]=[]
                event2album_train[event].append(album)
            else:
                if event not in event2album_val:
                    event2album_val[event]=[]
                event2album_val[event].append(album)
        #break
print('Train:',len(event2album_train))
total=0
for event in event2album_train:
    print(event,len(event2album_train[event]))
    total+=len(event2album_train[event])
print('total albumes:',total)
    
print('\nval:',len(event2album_val))
total=0
for event in event2album_val:
    print(event,len(event2album_val[event]))
    total+=len(event2album_val[event])
print('total albumes:',total)

1883
Train: 18
graduation 141
urbantrip 92
protest 38
show 117
birthday 153
christmas 88
museum 39
zoo 86
businessactivity 45
wedding 155
sports 57
architecture 43
cruise 39
personalsports 57
beachtrip 40
halloween 80
naturetrip 40
themepark 85
total albumes: 1395

val: 18
graduation 37
urbantrip 29
protest 11
show 22
businessactivity 8
christmas 19
museum 11
cruise 13
zoo 23
birthday 36
wedding 43
sports 15
architecture 6
halloween 29
personalsports 20
beachtrip 6
naturetrip 12
themepark 14
total albumes: 354


In [299]:
save_links_events(event2album_train,'/home/datasets/events/combined/filtered/train')
save_links_events(event2album_val,'/home/datasets/events/combined/filtered/val')

graduation 141
urbantrip 92
protest 38
show 117
birthday 153
christmas 88
museum 39
zoo 86
businessactivity 45
wedding 155
sports 57
architecture 43
cruise 39
personalsports 57
beachtrip 40
halloween 80
naturetrip 40
themepark 85
graduation 37
urbantrip 29
protest 11
show 22
businessactivity 8
christmas 19
museum 11
cruise 13
zoo 23
birthday 36
wedding 43
sports 15
architecture 6
halloween 29
personalsports 20
beachtrip 6
naturetrip 12
themepark 14


# WIDER

In [7]:
WIDER_DATA_DIR=ALL_DATA_DIR+'WIDER_v0.1/'
WIDER_TRAIN_DATA_DIR = WIDER_DATA_DIR+'full_res/train'
WIDER_VAL_DATA_DIR = WIDER_DATA_DIR+'full_res/val'

IMG_WIDER_DATA_DIR = ALL_DATA_DIR
WIDER_CATEGORIES=WIDER_DATA_DIR+'class_names.lst'
WIDER_TRAIN_FILE=WIDER_DATA_DIR+'train.lst'
WIDER_TEST_FILE=WIDER_DATA_DIR+'test.lst'

In [42]:
def getCategories(filename):
    ind2categories={}
    lines = [line.strip() for line in open(filename,'r')]
    for line in lines:
        row=line.split(' ')
        ind2categories[row[0]]=row[1]
    return ind2categories
ind2widerCategories=getCategories(WIDER_CATEGORIES)
print(ind2widerCategories)

{'50': 'Celebration_Or_Party', '29': 'Students_Schoolkids', '32': 'Worker_Laborer', '59': 'People_Driving_Car', '4': 'Dancing', '22': 'Picnic', '16': 'Award_Ceremony', '9': 'Press_Conference', '53': 'Raid', '60': 'Street_Battle', '45': 'Balloonist', '46': 'Jockey', '5': 'Car_Accident', '48': 'Parachutist_Paratrooper', '39': 'Ice_Skating', '17': 'Ceremony', '13': 'Interview', '1': 'Handshaking', '6': 'Funeral', '41': 'Swimming', '12': 'Group', '57': 'Angler', '0': 'Parade', '35': 'Basketball', '34': 'Baseball', '55': 'Sports_Coach_Trainer', '31': 'Waiter_Waitress', '10': 'People_Marching', '51': 'Dresses', '28': 'Sports_Fan', '14': 'Traffic', '15': 'Stock_Market', '21': 'Festival', '24': 'Soldier_Firing', '37': 'Soccer', '52': 'Photographers', '3': 'Riot', '18': 'Concerts', '8': 'Election_Campain', '20': 'Family_Group', '33': 'Running', '54': 'Rescue', '36': 'Football', '7': 'Cheering', '56': 'Voter', '58': 'Hockey', '43': 'Row_Boat', '11': 'Meeting', '44': 'Aerobics', '42': 'Car_Racing

In [43]:
def save_wider_links(filename, dir_to_save):
    lines = [line.strip() for line in open(filename,'r')]
    for row in log_progress(lines):
        file,category=row.split('jpg ')
        file+='jpg'
        src_file_path=os.path.join(IMG_WIDER_DATA_DIR,file)
        dst_file_path=os.path.join(dir_to_save,ind2widerCategories[category],os.path.basename(src_file_path))
        #print(src_file_path,dst_file_path)
        if not os.path.exists(dst_file_path):
            outdir_path=os.path.dirname(dst_file_path)
            if not os.path.exists(outdir_path):
                os.makedirs(outdir_path)
            os.symlink(src_file_path,dst_file_path)
        #break

In [44]:
save_wider_links(WIDER_TRAIN_FILE,WIDER_TRAIN_DATA_DIR)

In [48]:
save_wider_links(WIDER_TEST_FILE,WIDER_VAL_DATA_DIR)

# Culture event LAP

In [8]:
LAP_DATA_DIR=ALL_DATA_DIR+'culture_event_lap/dataset/'
LAP_TRAIN_DATA_DIR = LAP_DATA_DIR+'../full_res/train'
LAP_VAL_DATA_DIR = LAP_DATA_DIR+'../full_res/val'

IMG_LAP_TRAIN_DATA_DIR = LAP_DATA_DIR+'TRAIN/'
IMG_LAP_VAL_DATA_DIR = LAP_DATA_DIR+'VAL/'

In [236]:
TRAIN_SUFFIX='_train.txt'
VAL_SUFFIX='_val.txt'
LAP_categories=[f[:-len(TRAIN_SUFFIX)] for f in os.listdir(LAP_DATA_DIR) if f.endswith(TRAIN_SUFFIX)]
print(LAP_categories)

['Falles', 'Asakusa_Samba_Carnival', 'Heiva', 'AfrikaBurn', 'Sweden_Medieval_Week', 'Beltane_Fire', 'Junkanoo', 'Up_Helly_Aa', 'Australia_day', 'Eid_al-Adha', 'Eid_al-Fitr_Iraq', 'Quebec_Winter_Carnival', 'Harbin_Ice_and_Snow_Festival', 'Spice_Mas_Carnival', 'Cascamorras', 'Boston_Marathon', 'Pflasterspektakel', 'Carnaval_Dunkerque', 'Macys_Thanksgiving', 'Viking_Festival', 'Ballon_Fiesta', 'Desert_Festival_of_Jaisalmer', 'Krampusnacht', 'Fiesta_de_la_Candelaria', 'Tour_de_France', 'Cheongdo_Bullfighting_Festival', 'Keene_Pumpkin', 'Apokries', 'Bastille_day', 'Sahara_Festival', 'Helsinki_Samba_Carnaval', 'Timkat', 'Onbashira', 'Phi_Ta_Khon', 'Basel_Fasnacht', 'Annual_Buffalo_Roundup', 'Hajj', 'Carnival_Rio', 'Oktoberfest', 'Songkran_Water', 'Sapporo_Snow_Festival', 'Frozen_Dead_Guy_Days', 'San_Fermin', 'Correfocs', 'Los_Diablos_danzantes', 'Pushkar_Camel', 'Diada_de_Sant_Jordi', 'Carnival_of_Venice', 'Obon', 'Galungan', 'Magh_Mela', 'Sandfest', 'Tamborrada', 'Waysak_day', 'Bud_Billiken

In [96]:
def save_category(category,suffix,src_dir,dir_to_save):
    lines = [line.strip() for line in open(os.path.join(LAP_DATA_DIR,category+suffix),'r')]
    outdir_path=os.path.join(dir_to_save,category)
    if not os.path.exists(outdir_path):
        os.makedirs(outdir_path)
    for row in lines:
        filepath,is_category=row.split('\t')
        if is_category=='1':
            #print(filepath)
            src_file_path=os.path.join(src_dir,filepath)
            dst_file_path=os.path.join(outdir_path,filepath)
            #print(src_file_path,dst_file_path)
            if not os.path.exists(dst_file_path) and os.path.exists(src_file_path):
                os.symlink(src_file_path,dst_file_path)
                
for category in log_progress(LAP_categories):
    save_category(category,TRAIN_SUFFIX,IMG_LAP_TRAIN_DATA_DIR,LAP_TRAIN_DATA_DIR)
    save_category(category,VAL_SUFFIX,IMG_LAP_VAL_DATA_DIR,LAP_VAL_DATA_DIR)


In [237]:
def get_image2index(src_dir,suffix):
    lap_image2index={}
    ind=0
    for class_name in os.listdir(src_dir):
        if class_name in class_to_idx:
            y=class_to_idx[class_name]
            class_dir=os.path.join(src_dir,class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith('jpg'):
                    lap_image2index[img_name]=ind
                    ind+=1
    
    category2data={}
    for category in LAP_categories:
        lines = [line.strip() for line in open(os.path.join(LAP_DATA_DIR,category+suffix),'r')]
        category_data=[]
        for row in lines:
            filepath,is_category=row.split('\t')
            category_data.append((lap_image2index[filepath],1 if is_category=='1' else 0))
        category2data[category]=np.array(category_data)
    return lap_image2index,category2data

lap_image2index_train,category2data_train=get_image2index(LAP_TRAIN_DATA_DIR,TRAIN_SUFFIX)
lap_image2index_val,category2data_val=get_image2index(LAP_VAL_DATA_DIR,VAL_SUFFIX)

In [233]:
from sklearn.metrics import average_precision_score
def evaluate_lap(y_pred):
    mAP=0
    for category in LAP_categories:
        category_data=category2data_val[category]
        y_pred_category=y_pred[category_data[:,0]][:,class_to_idx[category]]
        y_true_category=category_data[:,1]
        #print(y_true_category[y_true_category==1], y_pred_category[y_true_category==1])
        ap=average_precision_score(y_true_category, y_pred_category)
        mAP+=ap
    mAP/=len(LAP_categories)
    return mAP

# Combined

In [9]:
COMBINED_DATA_DIR=ALL_DATA_DIR+'combined/filtered/'
COMBINED_TRAIN_DATA_DIR = COMBINED_DATA_DIR+'train'
COMBINED_VAL_DATA_DIR = COMBINED_DATA_DIR+'val'

In [145]:
from shutil import copyfile
for class_name in log_progress(os.listdir(WIDER_VAL_DATA_DIR)):
    class_dir=os.path.join(WIDER_VAL_DATA_DIR,class_name)        
    img_files=[img_name for img_name in os.listdir(class_dir) if img_name.lower().endswith('jpg')]
    shuffle(img_files)
    train_len=int(len(img_files)*0.5)
    print(class_dir,len(img_files),train_len)
    
    outdir_path=os.path.join(COMBINED_TRAIN_DATA_DIR,class_name)
    #print(outdir_path)
    #continue
    if not os.path.exists(outdir_path):
        os.makedirs(outdir_path)
    for img_name in img_files[:train_len]:
        dst_file_path=os.path.join(outdir_path,img_name)
        if not os.path.exists(dst_file_path):
            copyfile(os.path.join(WIDER_VAL_DATA_DIR,class_name,img_name),dst_file_path, follow_symlinks=False)
    
    outdir_path=os.path.join(COMBINED_VAL_DATA_DIR,class_name)
    if not os.path.exists(outdir_path):
        os.makedirs(outdir_path)
    for img_name in img_files[train_len:]:
        dst_file_path=os.path.join(outdir_path,img_name)
        #print(dst_file_path)
        if not os.path.exists(dst_file_path):
            copyfile(os.path.join(WIDER_VAL_DATA_DIR,class_name,img_name),dst_file_path, follow_symlinks=False)


/home/datasets/events/WIDER_v0.1/full_res/val/Car_Accident 652 326
/home/datasets/events/WIDER_v0.1/full_res/val/People_Driving_Car 354 177
/home/datasets/events/WIDER_v0.1/full_res/val/Car_Racing 668 334
/home/datasets/events/WIDER_v0.1/full_res/val/Soldier_Patrol 328 164
/home/datasets/events/WIDER_v0.1/full_res/val/Football 402 201
/home/datasets/events/WIDER_v0.1/full_res/val/Raid 334 167
/home/datasets/events/WIDER_v0.1/full_res/val/Dresses 392 196
/home/datasets/events/WIDER_v0.1/full_res/val/Spa 335 167
/home/datasets/events/WIDER_v0.1/full_res/val/Award_Ceremony 245 122
/home/datasets/events/WIDER_v0.1/full_res/val/Matador_Bullfighter 545 272
/home/datasets/events/WIDER_v0.1/full_res/val/Demonstration 1478 739
/home/datasets/events/WIDER_v0.1/full_res/val/Interview 714 357
/home/datasets/events/WIDER_v0.1/full_res/val/Street_Battle 234 117
/home/datasets/events/WIDER_v0.1/full_res/val/Running 259 129
/home/datasets/events/WIDER_v0.1/full_res/val/Sports_Fan 360 180
/home/dataset

In [20]:
import shutil

for key in ['Award_Ceremony','Balloonist','Baseball','Basketball', 'Car_Accident','Car_Racing', 'Ceremony','Cheering','Couple','Dresses', 'Election_Campain', 
            'Family_Group', 'Festival', 'Football', 'Greeting', 'Group', 'Handshaking', 'Hockey', 'Ice_Skating', 'Interview',
           'Jockey', 'Parachutist_Paratrooper', 'People_Driving_Car', 'People_Marching', 'Photographers', 'Press_Conference', 'Raid', 'Rescue', 'Riot', 
            'Row_Boat', 'Shoppers', 'Soccer', 'Soldier_Drilling', 'Soldier_Firing', 'Soldier_Patrol', 'Sports_Coach_Trainer', 'Sports_Fan', 'Stock_Market', 
            'Street_Battle', 'Students_Schoolkids', 'Swimming', 'Tennis', 'Traffic', 'Voter', 'Waiter_Waitress',
           'Viking_Festival','Sweden_Medieval_Week','Spice_Mas_Carnival','Apokries','Eid_al-Adha','Non-Class', 'Bud_Billiken', 'Fiesta_de_la_Candelaria','Basel_Fasnacht','Eid_al-Fitr_Iraq',
           
            'Ati-atihan', 'Battle_of_the_Oranges', 'Buenos_Aires_Tango_Festival', 'Carnaval_de_Oruro', 'Carnaval_Dunkerque', 'Carnevale_Di_Viareggio',
           'Desfile_de_Silleteros','Epiphany_greece', 'Festa_Della_Sensa', 'Festival_de_la_Marinera','Festival_of_the_Sun',
           'Frozen_Dead_Guy_Days','Grindelwald_Snow_Festival','Heiva','Kaapse_Klopse', 'Krampusnacht', 'Los_Diablos_danzantes',
           'Magh_Mela','Midsommar','Notting_hill_carnival','Onbashira','Passover','Pflasterspektakel', 'Queens_Day','Renaixement_Tortosa',
           'Sahara_Festival','San_Fermin','Tamborrada', 'Tapati_rapa_Nui', 'Timkat', 'Up_Helly_Aa','Waysak_day']:
    print (key)
    for split_dir in ['train','val']:
        dirpath=os.path.join(COMBINED_DATA_DIR,split_dir,key)
        if os.path.exists(dirpath):
            shutil.rmtree(dirpath)

Award_Ceremony
Balloonist
Baseball
Basketball
Car_Accident
Car_Racing
Ceremony
Cheering
Couple
Dresses
Election_Campain
Family_Group
Festival
Football
Greeting
Group
Handshaking
Hockey
Ice_Skating
Interview
Jockey
Parachutist_Paratrooper
People_Driving_Car
People_Marching
Photographers
Press_Conference
Raid
Rescue
Riot
Row_Boat
Shoppers
Soccer
Soldier_Drilling
Soldier_Firing
Soldier_Patrol
Sports_Coach_Trainer
Sports_Fan
Stock_Market
Street_Battle
Students_Schoolkids
Swimming
Tennis
Traffic
Voter
Waiter_Waitress
Viking_Festival
Sweden_Medieval_Week
Spice_Mas_Carnival
Apokries
Eid_al-Adha
Non-Class
Bud_Billiken
Fiesta_de_la_Candelaria
Basel_Fasnacht
Eid_al-Fitr_Iraq
Ati-atihan
Battle_of_the_Oranges
Buenos_Aires_Tango_Festival
Carnaval_de_Oruro
Carnaval_Dunkerque
Carnevale_Di_Viareggio
Desfile_de_Silleteros
Epiphany_greece
Festa_Della_Sensa
Festival_de_la_Marinera
Festival_of_the_Sun
Frozen_Dead_Guy_Days
Grindelwald_Snow_Festival
Heiva
Kaapse_Klopse
Krampusnacht
Los_Diablos_danzantes
Mag

In [62]:
UNSTABLE_DIR=os.path.join(COMBINED_DATA_DIR,'other')
for split_dir in ['train','val']:
    new_dir=os.path.join(UNSTABLE_DIR,split_dir)
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

THRESHOLD=20
for key, value in sorted(label2acc.items(), key=lambda kv: (kv[1][0],(kv[0],kv[1][1]))):
    if value[0]>=THRESHOLD:
        break
    print (key, value[0])
    for split_dir in ['train','val']:
        os.rename(os.path.join(COMBINED_DATA_DIR,split_dir,key), os.path.join(UNSTABLE_DIR,split_dir,key))

Sports_Coach_Trainer 4.046242774566474
Viking_Festival 4.166666666666667
Voter 5.0
Sweden_Medieval_Week 5.769230769230769
Spice_Mas_Carnival 6.818181818181818
Apokries 6.896551724137931
People_Marching 7.262569832402234
Election_Campain 7.547169811320755
Festival 7.558139534883721
Street_Battle 7.6923076923076925
Photographers 8.333333333333334
Riot 9.15032679738562
Cheering 9.655172413793103
Dia_de_los_Muertos 10.416666666666666
Halloween_Festival_of_the_Dead 10.909090909090908
Sports_Fan 11.11111111111111
Greeting 11.764705882352942
Eid_al-Adha 12.0
Non-Class 13.0
4_de_Julio 13.333333333333334
Interview 13.445378151260504
Bud_Billiken 13.461538461538462
Couple 13.934426229508198
St_Patrick_Day 14.285714285714286
Fiesta_de_la_Candelaria 14.583333333333334
Dancing 15.333333333333334
birthday 16.827852998065765
Basel_Fasnacht 17.77777777777778
Soldier_Drilling 17.821782178217823
Eid_al-Fitr_Iraq 17.857142857142858
Balloonist 19.444444444444443


# Train

In [10]:
from keras.models import Model,Sequential, load_model,model_from_json
from keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,resnet_v2,resnet
from keras.utils.generic_utils import CustomObjectScope
from keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
#from myimage import ImageDataGenerator

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

INPUT_SIZE = (224, 224)
net_model=mobilenet_v2
net_description='mobilenet2_alpha=1.4'
base_model_path='../scenes/places_mobilenet2_alpha=1.4_augm_ft_sgd_model.h5'

#net_description='mobilenet2_alpha=1.0'
#base_model_path='../scenes/places_mobilenet2_alpha=1.0_augm_ft_sgd_model.h5'

#net_model=resnet
#net_description='resnet101'
#base_model_path='../scenes/places_resnet_101_augm_ft_sgd.h5'

#net_model=densenet
#net_description='densenet121'
#net_model=mobilenet
#net_description='mobilenet'
#net_model=inception_resnet_v2
#net_description='inception_resnet_v2'

#net_model=inception_v3
#net_description='inception_v3'
#base_model_path='../scenes/places_inception_v3_augm_ft_sgd.h5'
#INPUT_SIZE = (299, 299)

#net_model=resnet_v2
#net_description='resnet2_50'

import efficientnet.keras as enet
net_model=enet
net_description='enet3'
base_model_path='../scenes/enet3_model_augm_ft_sgd.h5'
INPUT_SIZE = (300, 300)

if False:
    net_description='pec_'+net_description
    train_data_dir=PEC_TRAIN_DATA_DIR
    val_data_dir=PEC_VAL_DATA_DIR
elif False:
    net_description='cufed_'+net_description
    train_data_dir=CUFED_TRAIN_DATA_DIR
    val_data_dir=CUFED_VAL_DATA_DIR
elif False:
    net_description='wider_'+net_description
    train_data_dir=WIDER_TRAIN_DATA_DIR
    val_data_dir=WIDER_VAL_DATA_DIR
elif False:
    net_description='lap_'+net_description
    train_data_dir=LAP_TRAIN_DATA_DIR
    val_data_dir=LAP_VAL_DATA_DIR
else:
    net_description='combined_filtered_'+net_description
    train_data_dir=COMBINED_TRAIN_DATA_DIR
    val_data_dir=COMBINED_VAL_DATA_DIR
    

print(train_data_dir,val_data_dir)
#net_description+='_imgnet'
BATCH_SIZE=40 #512 #64 #32 #64

def save_model(model,filename):
    model.save_weights('weights.h5')
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)

    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights('weights.h5')
    loaded_model.save(filename)

Using TensorFlow backend.


/home/datasets/events/combined/filtered/train /home/datasets/events/combined/filtered/val


In [11]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.Session(config=config)
set_session(sess)

In [15]:
def random_crop(x, random_crop_size, sync_seed=None, **kwargs):
    np.random.seed(sync_seed)
    w, h = x.shape[1], x.shape[2]
    rangew = (w - random_crop_size[0]) // 2
    rangeh = (h - random_crop_size[1]) // 2
    offsetw = 0 if rangew == 0 else np.random.randint(rangew)
    offseth = 0 if rangeh == 0 else np.random.randint(rangeh)
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

preprocessing_function=net_model.preprocess_input
target_size=INPUT_SIZE
#preprocessing_function,target_size=my_preprocessing,(256,256)
print(target_size)

(300, 300)


In [16]:
train_datagen = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, 
                                   samplewise_std_normalization=False, zca_whitening=False, zca_epsilon=1e-06,
                                   rotation_range=0.1, width_shift_range=-0.1, height_shift_range=-0.1, brightness_range=None, shear_range=0.0, 
                                   zoom_range=0.1, channel_shift_range=1.05, fill_mode='nearest', horizontal_flip=True, vertical_flip=False,
                                   preprocessing_function=preprocessing_function)
train_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

tb = TensorBoard('logdir', write_graph=True, write_grads=True, write_images=True)


In [18]:
val_generator = test_datagen.flow_from_directory(
    val_data_dir, target_size=target_size,
    batch_size=BATCH_SIZE, class_mode='categorical',follow_links=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir, target_size=target_size,
    batch_size=BATCH_SIZE, class_mode='categorical',follow_links=True)


Found 24736 images belonging to 82 classes.


KeyboardInterrupt: 

In [19]:
N_CLASS=val_generator.num_classes
nb_train_samples=train_generator.samples
nb_validation_samples=val_generator.samples
print(N_CLASS,nb_train_samples,nb_validation_samples)

class_to_idx=val_generator.class_indices
idx_to_class={class_to_idx[cls]:cls for cls in class_to_idx}
print(idx_to_class)

#np.save('idx_to_class.npy',np.array(idx_to_class))

82 0 24736
{0: 'Aerobics', 1: 'AfrikaBurn', 2: 'Angler', 3: 'Annual_Buffalo_Roundup', 4: 'Aomori_Nebuta', 5: 'Asakusa_Samba_Carnival', 6: 'Ballon_Fiesta', 7: 'Beltane_Fire', 8: 'Birthday', 9: 'Boryeong_Mud', 10: 'Carnival_Rio', 11: 'Carnival_of_Venice', 12: 'Cascamorras', 13: 'Castellers', 14: 'Cheongdo_Bullfighting_Festival', 15: 'Chinese_New_Year', 16: 'Christmas', 17: 'Correfocs', 18: 'Crop_over', 19: 'Cruise', 20: 'Demonstration', 21: 'Desert_Festival_of_Jaisalmer', 22: 'Diada_de_Sant_Jordi', 23: 'Diwali_Festival_of_Lights', 24: 'Falles', 25: 'Funeral', 26: 'Galungan', 27: 'Gion_matsuri', 28: 'Graduation', 29: 'Gymnastics', 30: 'Hajj', 31: 'Halloween', 32: 'Harbin_Ice_and_Snow_Festival', 33: 'Helsinki_Samba_Carnaval', 34: 'Highland_Games', 35: 'Holi_Festival', 36: 'Infiorata_di_Genzano', 37: 'Junkanoo', 38: 'Keene_Pumpkin', 39: 'La_Tomatina', 40: 'Lewes_Bonfire', 41: 'Macys_Thanksgiving', 42: 'Maslenitsa', 43: 'Matador_Bullfighter', 44: 'Meeting', 45: 'Monkey_Buffet_Festival', 46: 

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

In [19]:
from keras.layers import LeakyReLU
def net(features_only=False):
    if True:
        base_model=load_model(base_model_path)
    else:
        input_shape=(INPUT_SIZE[0],INPUT_SIZE[1],3)
        #base_model = mobilenet.MobileNet(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
        #base_model = mobilenetv2.MobileNetV2(alpha=1.4, input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
        #base_model = resnet.ResNet101(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
        #base_model = densenet.DenseNet121(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
        #base_model = inception_resnet_v2.InceptionResNetV2(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
        base_model = inception_v3.InceptionV3(input_shape=input_shape, include_top=True, weights='imagenet')
    #base_model.summary()
    if features_only:
        x=base_model.layers[-2].output
    else:
        x=base_model.output

    #x = Dense(1024, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(N_CLASS, activation='softmax', use_bias=True,name='event_preds')(x)
    model=Model(base_model.inputs, x)
    return model,base_model

In [24]:
model,base_model=net(features_only=True)
start_epoch=0

for l in base_model.layers:
    l.trainable=False
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

descr=net_description+'_feats'
mc = ModelCheckpoint(descr+'.h5', monitor='val_acc', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_acc',patience=2)
FIRST_EPOCHS=10

print('Head tuning')
hist1=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=FIRST_EPOCHS, verbose=1, 
                    initial_epoch=0, callbacks=[tb, mc, es], validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________



Epoch 1/10
1884/1884 [==============================] - 643s 341ms/step - loss: 1.6651 - acc: 0.5101 - val_loss: 1.8222 - val_acc: 0.4831


Epoch 00001: val_acc improved from -inf to 0.48314, saving model to cufed_resnet101_feats.h5
Epoch 2/10
1884/1884 [==============================] - 637s 338ms/step - loss: 1.3910 - acc: 0.5867 - val_loss: 1.8616 - val_acc: 0.4830

Epoch 00002: val_acc did not improve from 0.48314
Epoch 3/10
1884/1884 [==============================] - 638s 339ms/step - loss: 1.2974 - acc: 0.6127 - val_loss: 1.8820 - val_acc: 0.4820

Epoch 00003: val_acc did not improve from 0.48314


In [25]:
start_epoch=len(hist1.history['loss'])
model.load_weights(descr+'.h5')
for l in base_model.layers:
    l.trainable=True
    
model.compile(optimizer=Adam(lr=1e-4,decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
mc = ModelCheckpoint(descr+'_ft.h5', monitor='val_acc', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_acc',patience=2 )
SECOND_EPOCHS=5+start_epoch

print('All tuning')
hist2=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, 
                    initial_epoch=start_epoch, callbacks=[mc,es], validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)


All tuning
Epoch 4/8
1884/1884 [==============================] - 1038s 551ms/step - loss: 1.2290 - acc: 0.6344 - val_loss: 1.7081 - val_acc: 0.5180

Epoch 00004: val_acc improved from -inf to 0.51799, saving model to cufed_resnet101_feats_ft.h5
Epoch 5/8
1884/1884 [==============================] - 999s 530ms/step - loss: 0.6964 - acc: 0.7951 - val_loss: 1.8967 - val_acc: 0.5050

Epoch 00005: val_acc did not improve from 0.51799
Epoch 6/8
1884/1884 [==============================] - 1000s 531ms/step - loss: 0.3024 - acc: 0.9209 - val_loss: 2.2777 - val_acc: 0.4950

Epoch 00006: val_acc did not improve from 0.51799


In [27]:
model.load_weights(descr+'_ft.h5')
model.compile(optimizer=SGD(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
mc = ModelCheckpoint(descr+'_ft_sgd.h5', monitor='val_acc', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_acc',patience=2)
second_epoch=start_epoch+len(hist2.history['loss'])
THIRD_EPOCHS=5+second_epoch

hist3=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=THIRD_EPOCHS, verbose=1, 
                    initial_epoch=second_epoch, callbacks=[mc,es], validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

Epoch 7/11
1884/1884 [==============================] - 996s 528ms/step - loss: 0.7402 - acc: 0.7966 - val_loss: 1.7113 - val_acc: 0.5262

Epoch 00007: val_acc improved from -inf to 0.52619, saving model to cufed_resnet101_ft_sgd.h5
Epoch 8/11
1884/1884 [==============================] - 967s 513ms/step - loss: 0.7185 - acc: 0.8009 - val_loss: 1.7208 - val_acc: 0.5262

Epoch 00008: val_acc improved from 0.52619 to 0.52621, saving model to cufed_resnet101_ft_sgd.h5
Epoch 9/11
1884/1884 [==============================] - 967s 513ms/step - loss: 0.7044 - acc: 0.8044 - val_loss: 1.7305 - val_acc: 0.5271

Epoch 00009: val_acc improved from 0.52621 to 0.52714, saving model to cufed_resnet101_ft_sgd.h5
Epoch 10/11
1884/1884 [==============================] - 968s 514ms/step - loss: 0.6925 - acc: 0.8064 - val_loss: 1.7362 - val_acc: 0.5259

Epoch 00010: val_acc did not improve from 0.52714
Epoch 11/11
1884/1884 [==============================] - 971s 515ms/step - loss: 0.6824 - acc: 0.8089 - v

In [245]:
#featureModel=load_model('lap_inception_v3_feats_ft_sgd.h5') #_ft_sgd
featureModel=load_model('lap_mobilenet2_alpha=1.4_feats_ft_sgd.h5') # #combined_filtered
y_features_val_fc_proba=[]
for class_name in log_progress(os.listdir(val_data_dir)):
    if class_name in class_to_idx:
        class_dir=os.path.join(val_data_dir,class_name)
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith('jpg'):
                img=cv2.imread(os.path.join(class_dir,img_name))
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                img=cv2.resize(img,INPUT_SIZE)
                inp = net_model.preprocess_input(np.expand_dims(img, axis=0).astype(np.float32))
                scores=featureModel.predict(inp)
                y_features_val_fc_proba.append(scores)
    
y_features_val_fc_proba= np.concatenate(y_features_val_fc_proba, axis=0)

In [49]:
def get_model_scores(featureModel,data_dir):
    y_features_fc_proba,y_correct=[],[]
    images=[]
    for class_name in log_progress(os.listdir(data_dir)):
        if class_name in class_to_idx:
            y=class_to_idx[class_name]
            class_dir=os.path.join(data_dir,class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith('jpg'):
                    img=cv2.imread(os.path.join(class_dir,img_name))
                    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                    img=cv2.resize(img,INPUT_SIZE)
                    images.append(img.astype(np.float32))
                    y_correct.append(y)
                    if len(images)>=BATCH_SIZE:
                        inp = net_model.preprocess_input(np.array(images))
                        scores=featureModel.predict(inp)
                        y_features_fc_proba.append(scores)
                        images=[]
    if len(images)>0:
        inp = net_model.preprocess_input(np.array(images))
        scores=featureModel.predict(inp)
        y_features_fc_proba.append(scores)
    y_features_fc_proba= np.concatenate(y_features_fc_proba, axis=0)
    y_correct=np.array(y_correct)
    print(y_features_fc_proba.shape,y_correct.shape)
    return y_features_fc_proba,y_correct

#FT_MODEL_NAME='cufed_mobilenet2_alpha=1.4_feats_ft_sgd'
FT_MODEL_NAME='pec_resnet101_feats_ft'
#FT_MODEL_NAME=net_description+'_feats_ft_sgd'
print(FT_MODEL_NAME)
featureModel=load_model(FT_MODEL_NAME+'.h5')

y_features_val_fc_proba,y_val_correct=get_model_scores(featureModel,val_data_dir)
y_features_train_fc_proba,y_train_correct=get_model_scores(featureModel,train_data_dir)
np.savez(FT_MODEL_NAME+'_scores.npz', y_features_train_fc_proba,y_train_correct,y_features_val_fc_proba,y_val_correct)

pec_resnet101_feats_ft


(11085, 14) (11085,)


(50279, 14) (50279,)


In [50]:
print(y_features_val_fc_proba.shape, y_val.shape)
y_features_val_fc=np.argmax(y_features_val_fc_proba,axis=1)
print('total acc=',100.0*(y_val==y_features_val_fc).sum()/len(y_val), compute_accuracy(y_features_val_fc))

(11085, 14) (11085,)
total acc= 61.44339197113216 61.44339197113216


# Classifiers

In [20]:
_,base_model=net()
model=Model(base_model.inputs, [base_model.layers[-2].output,base_model.output])
model.summary()










Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 150, 150, 40) 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 150, 150, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 150, 150, 40) 0           s

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [21]:
X_features_val,X_scores_val,y_val=[],[],[]
for class_name in log_progress(os.listdir(val_data_dir)):
    if class_name in class_to_idx:
        y=class_to_idx[class_name]
        class_dir=os.path.join(val_data_dir,class_name)
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith('jpg'):
                img=cv2.imread(os.path.join(class_dir,img_name))
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                img=cv2.resize(img,INPUT_SIZE)
                inp = net_model.preprocess_input(np.expand_dims(img, axis=0).astype(np.float32))
                features,scores=model.predict(inp)
                #print(features.shape,scores.shape)
                X_features_val.append(features)
                X_scores_val.append(scores)
                y_val.append(y)
    
X_features_val= np.concatenate(X_features_val, axis=0)
X_scores_val=np.concatenate(X_scores_val, axis=0)
y_val=np.array(y_val)
print(X_features_val.shape, X_scores_val.shape,y_val.shape)

X_features_train,X_scores_train,y_train=[],[],[]
for class_name in log_progress(os.listdir(train_data_dir)):
    if class_name in class_to_idx:
        y=class_to_idx[class_name]
        class_dir=os.path.join(train_data_dir,class_name)
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith('jpg'):
                img=cv2.imread(os.path.join(class_dir,img_name))
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                img=cv2.resize(img,INPUT_SIZE)
                inp = net_model.preprocess_input(np.expand_dims(img, axis=0).astype(np.float32))
                features,scores=model.predict(inp)
                X_features_train.append(features)
                X_scores_train.append(scores)
                y_train.append(y)
X_features_train= np.concatenate(X_features_train, axis=0)
X_scores_train=np.concatenate(X_scores_train, axis=0)
y_train=np.array(y_train)
print(X_features_train.shape, X_scores_train.shape,y_train.shape)

(24736, 1536) (24736, 431) (24736,)


(87440, 1536) (87440, 431) (87440,)


In [54]:
USE_EFFICIENT_NET=True
if USE_EFFICIENT_NET:
    from efficient_net import EfficientNet
    ckpt_dir=None
    #model_name='efficientnet-b0'
    ckpt_dir='efficientnet-b5-randaug'
    net=EfficientNet(model_name=model_name, ckpt_dir=ckpt_dir)
    
    X_features_val,X_scores_val,y_val=[],[],[]
    for class_name in log_progress(os.listdir(val_data_dir)):
        if class_name in class_to_idx:
            y=class_to_idx[class_name]
            class_dir=os.path.join(val_data_dir,class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith('jpg'):
                    features=net.extract_features(os.path.join(class_dir,img_name))
                    X_features_val.append(features)
                    y_val.append(y)

    X_features_val= np.concatenate(X_features_val, axis=0)
    y_val=np.array(y_val)
    print(X_features_val.shape, y_val.shape)

    X_features_train,X_scores_train,y_train=[],[],[]
    for class_name in log_progress(os.listdir(train_data_dir)):
        if class_name in class_to_idx:
            y=class_to_idx[class_name]
            class_dir=os.path.join(train_data_dir,class_name)
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith('jpg'):
                    features=net.extract_features(os.path.join(class_dir,img_name))
                    X_features_train.append(features)
                    y_train.append(y)
    X_features_train= np.concatenate(X_features_train, axis=0)
    y_train=np.array(y_train)
    print(X_features_train.shape,y_train.shape)

INFO:tensorflow:Restoring parameters from /opt/lab/tpu/models/official/efficientnet/checkpoints/efficientnet-b0/model.ckpt


INFO:tensorflow:Restoring parameters from /opt/lab/tpu/models/official/efficientnet/checkpoints/efficientnet-b0/model.ckpt


(19420, 1280) (19420,)


(75377, 1280) (75377,)


In [23]:
X_features_train,X_scores_train,y_train=[],[],[]
for _ in range(nb_train_samples // BATCH_SIZE):
    X_batch,y_batch=train_generator.next()
    features,scores=model.predict(X_batch)
    X_features_train.append(features)
    X_scores_train.append(scores)
    y_train.append(y_batch)
X_features_train= np.concatenate(X_features_train, axis=0)
X_scores_train=np.concatenate(X_scores_train, axis=0)
y_train=np.concatenate(y_train, axis=0)
print(X_features_train.shape, X_scores_train.shape,y_train.shape)

(50272, 2048) (50272, 1000) (50272, 14)


In [22]:
class_to_idx_ignore_case={cls.lower():class_to_idx[cls] for cls in class_to_idx}
print(class_to_idx_ignore_case)

y_val_multilabel=[]
for class_name in log_progress(os.listdir(val_data_dir)):
    if class_name in class_to_idx:
        y=class_to_idx[class_name]
        class_dir=os.path.join(val_data_dir,class_name)
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith('jpg'):
                y_multilabel=np.zeros(N_CLASS)
                if train_data_dir==CUFED_TRAIN_DATA_DIR:
                    img_labels=img_name.split('_')[2].split(',')
                    for img_label in img_labels:
                        y_multilabel[class_to_idx[img_label]]=1
                elif train_data_dir==COMBINED_TRAIN_DATA_DIR:
                    name_splitted=img_name.split('_')
                    img_labels=[class_name.lower()] if len(name_splitted)<3 else name_splitted[2].lower().split(',')
                    for img_label in img_labels:
                        if img_label in class_to_idx_ignore_case:
                            y_multilabel[class_to_idx_ignore_case[img_label]]=1
                else:
                    y_multilabel[y]=1
                y_val_multilabel.append(y_multilabel)

y_val_multilabel=np.array(y_val_multilabel)
print(y_val_multilabel.shape,np.argmax(y_val_multilabel,axis=1))

y_train_multilabel=[]
for class_name in log_progress(os.listdir(train_data_dir)):
    if class_name in class_to_idx:
        y=class_to_idx[class_name]
        class_dir=os.path.join(train_data_dir,class_name)
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith('jpg'):
                y_multilabel=np.zeros(N_CLASS)
                if train_data_dir==CUFED_TRAIN_DATA_DIR:
                    img_labels=img_name.split('_')[2].split(',')
                    for img_label in img_labels:
                        y_multilabel[class_to_idx[img_label]]=1
                elif train_data_dir==COMBINED_TRAIN_DATA_DIR:
                    name_splitted=img_name.split('_')
                    img_labels=[class_name.lower()] if len(name_splitted)<3 else name_splitted[2].lower().split(',')
                    for img_label in img_labels:
                        if img_label in class_to_idx_ignore_case:
                            y_multilabel[class_to_idx_ignore_case[img_label]]=1
                else:
                    y_multilabel[y]=1
                y_train_multilabel.append(y_multilabel)

y_train_multilabel=np.array(y_train_multilabel)
print(y_train_multilabel.shape,np.argmax(y_train_multilabel,axis=1))

{'urbantrip': 80, 'wedding': 69, 'maslenitsa': 42, 'pingxi_lantern_festival': 53, 'pushkar_camel': 54, 'zoo': 81, 'phi_ta_khon': 51, 'aerobics': 0, 'cruise': 19, 'oktoberfest': 49, 'monkey_buffet_festival': 45, 'junkanoo': 37, 'thrissur_pooram': 65, 'chinese_new_year': 15, 'running': 57, 'songkran_water': 60, 'lewes_bonfire': 40, 'themepark': 79, 'carnival_of_venice': 11, 'thaipusam': 64, 'falles': 24, 'asakusa_samba_carnival': 5, 'worker_laborer': 70, 'birthday': 8, 'sandfest': 58, 'graduation': 28, 'tour_de_france': 67, 'desert_festival_of_jaisalmer': 21, 'afrikaburn': 1, 'architecture': 71, 'naadam_festival': 47, 'beltane_fire': 7, 'boryeong_mud': 9, 'personalsports': 75, 'carnival_rio': 10, 'sapporo_snow_festival': 59, 'highland_games': 34, 'helsinki_samba_carnaval': 33, 'aomori_nebuta': 4, 'harbin_ice_and_snow_festival': 32, 'diada_de_sant_jordi': 22, 'protest': 76, 'galungan': 26, 'la_tomatina': 39, 'correfocs': 17, 'funeral': 25, 'christmas': 16, 'castellers': 13, 'ballon_fiesta

(24736, 82) [59 59 59 ... 27 27 27]


(87440, 82) [59 59 59 ... 27 27 27]


In [23]:
if train_data_dir in [CUFED_TRAIN_DATA_DIR,COMBINED_TRAIN_DATA_DIR]:
    def compute_accuracy(y_pred):
        if len(y_pred.shape)==1:
            y_pred_ohe=np.eye(N_CLASS)[y_pred]
        else:
            y_pred_ohe=y_pred
        #print(y_pred_ohe.shape, y_val_multilabel.shape)
        return 100.0*(np.max(np.logical_and(y_val_multilabel, y_pred_ohe),axis=1)).sum()/len(y_pred)
else:
    def compute_accuracy(y_pred):
        return 100.0*(y_val==y_pred).sum()/len(y_val)

In [22]:
classifiers=[('rf100',RandomForestClassifier(n_estimators=100)),#('rf1000',RandomForestClassifier(n_estimators=1000)),
             #('erf100',ExtraTreesClassifier(n_estimators=100)),#('erf1000',ExtraTreesClassifier(n_estimators=1000)),
             ('linear svm',LinearSVC()),
             #('Svm (linear)',SVC(kernel='linear',gamma='scale')),
             #('Svm (rbf)',SVC(kernel='rbf',gamma='scale'))
            ]

In [58]:
print('Features')
for cls_name,classifier in classifiers:
    classifier.fit(X_features_train,y_train)
    y_val_pred=classifier.predict(X_features_val)
    acc=100.0*(y_val==y_val_pred).sum()/len(y_val)
    print(cls_name,' acc=',acc, compute_accuracy(y_val_pred))

Features
rf100  acc= 46.05046343975283 50.4994850669413
linear svm  acc= 49.860968074150364 54.531410916580846


/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [81]:
print('Scores')
for cls_name,classifier in classifiers:
    classifier.fit(X_scores_train,y_train)
    y_val_pred=classifier.predict(X_scores_val)
    acc=100.0*(y_val==y_val_pred).sum()/len(y_val)
    print(cls_name,' acc=',acc, compute_accuracy(y_val_pred))

Scores
rf100  acc= 46.97734294541709 51.27188465499485
linear svm  acc= 40.458290422245106 44.72193614830072


In [57]:
#classifiers=[('rf',RandomForestClassifier(n_estimators=100)),('linear svm',LinearSVC())]
print('Normalized features')
for cls_name,classifier in classifiers:
    classifier.fit(preprocessing.normalize(X_features_train,norm='l2'),y_train)
    y_val_pred=classifier.predict(preprocessing.normalize(X_features_val,norm='l2'))
    acc=100.0*(y_val==y_val_pred).sum()/len(y_val)
    print(cls_name,' acc=',acc, compute_accuracy(y_val_pred))

Normalized features
rf100  acc= 46.627188465499486 50.94232749742533
linear svm  acc= 51.07106076210093 55.7826982492276


In [22]:
#Results per category
classifier=LinearSVC()
if True:
    classifier.fit(preprocessing.normalize(X_features_train,norm='l2'),y_train)
    y_val_pred=classifier.predict(preprocessing.normalize(X_features_val,norm='l2'))
else:
    classifier.fit(preprocessing.normalize(X_features_train-X_features_train.mean(axis=1, keepdims=True),norm='l2'),y_train)
    y_val_pred=classifier.predict(preprocessing.normalize(X_features_val-X_features_val.mean(axis=1, keepdims=True),norm='l2'))

In [23]:
y_val_pred=classifier.predict(preprocessing.normalize(X_features_val,norm='l2'))

In [26]:
print('total acc=',100.0*(y_val==y_val_pred).sum()/len(y_val), compute_accuracy(y_val_pred))

total acc= 58.10155239327296 61.07697283311772


In [27]:
y_labels=np.unique(y_val)
label2acc={}
for y_label in y_labels:
    y_pred=y_val_pred[y_val==y_label]
    acc=100.0*(y_label==y_pred).sum()/len(y_pred)
    label2acc[idx_to_class[y_label]]=(acc,len(y_pred))

for key, value in sorted(label2acc.items(), key=lambda kv: (kv[1][0],(kv[0],kv[1][1])),reverse=True):
    print (key, value[1], value[0])

La_Tomatina 49 93.87755102040816
Pingxi_Lantern_Festival 45 91.11111111111111
Infiorata_di_Genzano 47 89.36170212765957
Sandfest 47 87.23404255319149
Harbin_Ice_and_Snow_Festival 55 85.45454545454545
Boryeong_Mud 60 85.0
Junkanoo 58 84.48275862068965
Matador_Bullfighter 273 83.51648351648352
Falles 72 81.94444444444444
Monkey_Buffet_Festival 37 81.08108108108108
Vancouver_Symphony_of_Fire 42 80.95238095238095
AfrikaBurn 84 80.95238095238095
Spa 168 80.35714285714286
Sapporo_Snow_Festival 45 80.0
Keene_Pumpkin 55 80.0
Correfocs 50 80.0
Demonstration 739 79.70230040595399
Ballon_Fiesta 54 79.62962962962963
Macys_Thanksgiving 47 78.72340425531915
Phi_Ta_Khon 50 78.0
Thrissur_Pooram 67 77.61194029850746
Gymnastics 194 76.80412371134021
Carnival_Rio 56 76.78571428571429
Holi_Festival 51 76.47058823529412
Aomori_Nebuta 80 76.25
zoo 1077 75.67316620241411
Picnic 175 75.42857142857143
Surgeons 101 75.24752475247524
Pushkar_Camel 48 75.0
Lewes_Bonfire 44 75.0
Castellers 64 75.0
Asakusa_Samba_Ca

In [28]:
for y_label in y_labels:
    y_pred=y_val_pred[y_val==y_label]
    mistakes=y_pred[y_label!=y_pred]
    incorrect_labels={}
    delta=100/len(y_pred)
    for ind in mistakes:
        l=idx_to_class[ind]
        if l in incorrect_labels:
            incorrect_labels[l]+=delta
        else:
            incorrect_labels[l]=delta
    #label2acc[]=(acc,len(y_pred))

    top_incorrect=sorted(incorrect_labels.items(), key=lambda kv: kv[1],reverse=True)
    acc=100.0*(y_label==y_pred).sum()/len(y_pred)
    #for key, value in :
    if acc-20<top_incorrect[0][1]:
        print (idx_to_class[y_label],acc,top_incorrect[:2])

## Convert linear svm classifier to pb

In [24]:
from keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,AveragePooling2D, Activation, Conv2D, Lambda, Input, Reshape
from keras import backend as K

layer=base_model.layers[-2].output
print(layer)
l2_norm = Lambda(lambda  x: K.l2_normalize(x,axis=1))(layer)
event_scores = Dense(N_CLASS,weights=[classifier.coef_.T,classifier.intercept_],name='event_fc')(l2_norm)
new_model=Model(base_model.inputs, [base_model.output,event_scores])
new_model.summary()

Tensor("top_dropout/cond/Merge:0", shape=(?, 1536), dtype=float32)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 150, 150, 40) 1080        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 150, 150, 40) 160         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 150, 150, 40) 0           stem_bn[0][0]                    
_________________________

In [12]:
#MODEL_PATH='places_event_mobilenet2_alpha=1.0_augm_ft_sgd_model'
#preprocessing_function=mobilenet_v2.preprocess_input

#MODEL_PATH='places_event_inception_v3_augm_ft_sgd_model'
#preprocessing_function=inception_v3.preprocess_input

MODEL_PATH='places_event_enet3_augm_ft_sgd_model'
preprocessing_function=enet.preprocess_input

In [26]:
save_model(new_model,MODEL_PATH+'.h5')

## FC layer to features 

In [80]:
try:
    del feature_classifier_model
except:
    pass

if False:
    from keras.layers import concatenate
    input1 = Input(shape=(X_features_train.shape[1],))
    input2 = Input(shape=(X_scores_train.shape[1],))
    x=concatenate([input1,input2])
    x=Dense(1024, activation='relu')(x)
    x=Dropout(0.25)(x)
    event_preds=Dense(N_CLASS, activation='softmax', use_bias=True,name='event_preds')(x)
    feature_classifier_model=Model(inputs=[input1,input2],outputs=[event_preds])
else:
    feature_classifier_model=Sequential()
    if True:
        feature_classifier_model.add(Dense(1024, input_shape=X_features_train.shape[1:], activation='relu'))
        #feature_classifier_model.add(LeakyReLU())
        feature_classifier_model.add(Dropout(0.25))
        feature_classifier_model.add(Dense(N_CLASS, activation='softmax', use_bias=True,name='event_preds'))
    else:
        feature_classifier_model.add(Dense(N_CLASS, input_shape=X_features_train.shape[1:], activation='softmax', use_bias=True,name='event_preds'))

feature_classifier_model.compile(optimizer=Adam(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
feature_classifier_model.summary()

es=EarlyStopping(monitor='val_acc',patience=2)
#mc = ModelCheckpoint(OBJECT_CLASSIFIER_NAME, monitor='val_acc', verbose=1, save_best_only=True)
                
feature_classifier_model.fit(X_features_train,y_train, batch_size=64, epochs=5, verbose=1, callbacks=[es], validation_data=(X_features_val,y_val)) #,class_weight=class_weights)
#feature_classifier_model.fit([X_features_train,X_scores_train],y_train, batch_size=64, epochs=10, verbose=1, callbacks=[es], validation_data=([X_features_val,X_scores_val],y_val))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              1836032   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
event_preds (Dense)          (None, 23)                23575     
Total params: 1,859,607
Trainable params: 1,859,607
Non-trainable params: 0
_________________________________________________________________
Train on 75377 samples, validate on 19420 samples
Epoch 1/5
75377/75377 [==============================] - 11s 149us/step - loss: 1.5849 - acc: 0.5220 - val_loss: 1.7185 - val_acc: 0.4931
Epoch 2/5
75377/75377 [==============================] - 6s 79us/step - loss: 1.3297 - acc: 0.5952 - val_loss: 1.7782 - val_acc: 0.4873
Epoch 3/5
75377/75377 [===========================

In [81]:
feature_classifier_model.compile(optimizer=SGD(lr=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

feature_classifier_model.fit(X_features_train,y_train, batch_size=64, epochs=5, verbose=1, validation_data=(X_features_val,y_val))

Train on 75377 samples, validate on 19420 samples
Epoch 1/5
75377/75377 [==============================] - 11s 141us/step - loss: 1.0569 - acc: 0.6780 - val_loss: 1.7811 - val_acc: 0.4908
Epoch 2/5
75377/75377 [==============================] - 5s 64us/step - loss: 1.0398 - acc: 0.6855 - val_loss: 1.7782 - val_acc: 0.4911
Epoch 3/5
75377/75377 [==============================] - 5s 66us/step - loss: 1.0350 - acc: 0.6879 - val_loss: 1.7773 - val_acc: 0.4912
Epoch 4/5
75377/75377 [==============================] - 5s 65us/step - loss: 1.0292 - acc: 0.6884 - val_loss: 1.7774 - val_acc: 0.4923
Epoch 5/5
75377/75377 [==============================] - 5s 66us/step - loss: 1.0256 - acc: 0.6902 - val_loss: 1.7773 - val_acc: 0.4923
